In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
from requests_html import HTMLSession

In [4]:
url = 'https://www.cnnindonesia.com/search?query=covid'
links = []

options = webdriver.ChromeOptions()
pathToChromeDriver = "C:/Program Files/Google/Chrome/Application/chromedriver.exe"
browser = webdriver.Chrome(executable_path=pathToChromeDriver,
                           options=options)
    
try:
    browser.get(url)
    browser.implicitly_wait(10)
    html = browser.page_source
    
    num_btn_click = 145
    for i in range(1, num_btn_click):
        nxt_btn = browser.find_element_by_xpath('//*[@id="content"]/div/div[2]/section/div/div/div/a').click()
        time.sleep(2)
    
    content = browser.find_element_by_class_name('media_rows')
    for elt in content.find_elements_by_tag_name('article'):
        
        link = elt.find_element_by_tag_name('a')
        href = link.get_attribute('href')
        links.append(href)
            
            
finally:
    browser.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <a class="active" href="/search/?query=covid">...</a> is not clickable at point (398, 199). Other element would receive the click: <span class="ratiobox_content lqd imgLiquid_bgSize imgLiquid_ready" style="background-image: url(&quot;https://akcdn.detik.net.id/visual/2021/08/31/ilustrasi-perempuan-belanja-sayur-di-pasar-tradisional_169.jpeg?w=220&amp;q=90&quot;); background-size: cover; background-position: center top; background-repeat: no-repeat;">...</span>
  (Session info: chrome=103.0.5060.114)


In [3]:
len(links)

1015

In [4]:
data = []

for link in links:
    url = requests.get(link)
    soup = bs(url.text, 'lxml')
    cont = soup.find('div', class_='content_detail')
    title = cont.h1.get_text(strip=True) if cont.h1 else 'no title'
    date = cont.find('div', class_='date').text.strip() if cont.find('div', class_='date') else 'no date'
    content = ' '.join([p.get_text().strip() for p in cont.select('p')]) if cont.p else 'no content'    

    data.append({
        'Title':title,
        'Text':content,
        'Date':date
    })    
        

In [5]:
df = pd.DataFrame(data)
df.to_csv('C:/Users/zaliz/OneDrive/Desktop/scraping baru/crawl_cnn.csv', index=False)

In [6]:
cnn_dt = pd.read_csv('C:/Users/zaliz/OneDrive/Desktop/scraping baru/crawl_cnn.csv')
cnn_dt.head(10)

,judul,teks,tanggal
0,"Indeks Persepsi Korupsi, RI Urutan 102 Dunia S...",Indonesia berada di peringkat 102 dunia negara...,"Kamis, 09 Dec 2021 09:50 WIB"
1,Alasan Tepat Beli Rumah di Akhir Tahun,Punya rumah adalah impian bagi semua orang. Ba...,"Kamis, 09 Dec 2021 09:40 WIB"
2,Rupiah Menguat ke Rp14.319 per Dolar AS Pagi Ini,Nilai tukar rupiah berada di posisi Rp14.319 p...,"Kamis, 09 Dec 2021 09:10 WIB"
3,Satgas Sebut Pemerintah Bangun Kewaspadaan Mas...,Ketua Bidang Penanganan Kesehatan Satgas Covid...,"Kamis, 09 Dec 2021 09:00 WIB"
4,"Harga Emas Antam Hari Ini 9 Desember, Turun ke...",Harga jual emas PT Aneka Tambang (Persero) Tbk...,"Kamis, 09 Dec 2021 09:00 WIB"
5,Angka Kematian Akibat Covid di Jerman Kembali ...,Angka kematian harian akibat Covid-19 di Jerma...,"Kamis, 09 Dec 2021 08:50 WIB"
6,Suara Dua Bakal Calon Ketum PBNU: Said Aqil da...,Gelaran muktamar ke-34 Nahdlatul Ulama (NU) te...,"Kamis, 09 Dec 2021 08:31 WIB"
7,"Profil Olaf Scholz, Kanselir Baru Jerman Pengg...",Olaf Scholz resmi menjadi kanselir baru Jerman...,"Kamis, 09 Dec 2021 08:25 WIB"
8,VIDEO: Penggunaan Masker dengan Benar,"Pandemi covid 19 yang belum usai, membuat masy...","CNN Indonesia TV, CNN Indonesia | Kamis, 09 De..."
9,"Hukuman Mati Koruptor, Antara Efek Jera dan Ha...","Terdakwa kasus dugaan korupsi PT ASABRI, Heru ...","Kamis, 09 Dec 2021 08:00 WIB"


In [8]:
len(cnn_dt)

1015